In [21]:
import sqlalchemy
import pandas as pd
from binance.client import Client

In [22]:
client = Client(api_key="ptemDSB2tPEyEyA2IXpdmgNZenhkWOJ1ZTffmXuT52lRrMJ7yCMjxAiPDqbXIuyT",api_secret="m1mxb2nelryRmplgCPVFllHEam6LllWkpFX2T2LkXyInzWDR4B0Iw0AV9kaLCh7Z")

In [23]:
engine = sqlalchemy.create_engine('sqlite:///GMTUSDTstream.db')

In [24]:
df = pd.read_sql('GMTUSDT',engine)

In [ ]:
#Trendfollowing
#strategy: if the crypto was rising by x%->Buy
#exit when profit is above 0.15% or loss is crossing -0.15%

In [ ]:
#look
def strategy(entry,lookback,qty,open_position=False):
    while True:
        df = pd.read_sql('GMTUSDT',engine)
        lookbackperiod=df.iloc[-lookback:]
        cumulationretrun=(lookbackperiod.Price.pct_change()+1).cumprod()-1
        if not open_position:
            if cumulationretrun[cumulationretrun.last_valid_index()]>entry:
                order = client.create_order(symbol='GMTUSDT',side='BUY',type='MARKET',quantity=qty)
                print(order)
                open_position=True
                break
    if open_position:
        while True:
            df=pd.read_sql('GMTUSDT',engine)
            sincebuy= df.loc[df.Time > pd.to_datetime(order['transactTime'],unit='ms')]
            if len(sincebuy) > 1:
                sincebuyret=(sincebuy.Price.pct_change()+1).cumprod()-1
                lastentry=sincebuyret[sincebuy.last_valid_index()]
                if lastentry > 0.0015 or lastentry < -0.0015:
                      order = client.create_order(symbol='GMTUSDT',side='BUY',type='MARKET',quantity=qty)
                      print(order)
                      break